![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [342]:
import pandas as pd
import numpy as np

df = pd.read_csv("bank_marketing.csv")
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [343]:
client_columns = ["client_id","age","job","marital","education","credit_default","mortgage"]
campaign_columns= ["client_id","number_contacts","contact_duration","previous_campaign_contacts",
                   "previous_outcome","campaign_outcome"]
economics_columns = ["client_id","cons_price_idx","euribor_three_months"]
client = df[client_columns]
campaign = df[campaign_columns]
economics = df[economics_columns]

In [344]:
client_data.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [345]:
client["job"] = client["job"].str.replace('.', '_')
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin_,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [346]:
client["education"] = client["education"].str.replace('.','_')
client.loc[df["education"]=='unknown','education'] = np.NaN
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,no,no
1,1,57,services,married,high_school,unknown,no
2,2,37,services,married,high_school,no,yes
3,3,40,admin_,married,basic_6y,no,no
4,4,56,services,married,high_school,no,no


In [347]:
client["credit_default"].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [348]:
client["credit_default"] = client["credit_default"].map({"unknown": False, "yes": True, "no": False})

In [349]:
client["credit_default"].unique()

array([False,  True])

In [350]:
client["mortgage"].unique()

array(['no', 'yes', 'unknown'], dtype=object)

In [351]:
client["mortgage"] = client["mortgage"].map({"unknown": False, "yes": True, "no": False}).astype(bool)
client["mortgage"].unique()

array([False,  True])

In [352]:
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB


In [353]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,nonexistent,no
1,1,1,149,0,nonexistent,no
2,2,1,226,0,nonexistent,no
3,3,1,151,0,nonexistent,no
4,4,1,307,0,nonexistent,no


In [354]:
campaign["previous_outcome"].unique()

array(['nonexistent', 'failure', 'success'], dtype=object)

In [355]:
campaign["previous_outcome"] = campaign["previous_outcome"]\
.map({"nonexistent": False, "success": True, "failure": False})
campaign["previous_outcome"].unique()

array([False,  True])

In [356]:
campaign["campaign_outcome"].unique()

array(['no', 'yes'], dtype=object)

In [357]:
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": True, "no": False})
campaign["campaign_outcome"].unique()

array([False,  True])

In [358]:
campaign_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [359]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,False,False
1,1,1,149,0,False,False
2,2,1,226,0,False,False
3,3,1,151,0,False,False
4,4,1,307,0,False,False


In [360]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [361]:
df["month"] = df['month'].str.capitalize()
df["year"] = "2022"
df["day"] = df["day"].astype('str') 
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year
0,0,56,housemaid,married,basic.4y,no,no,May,13,261,1,0,nonexistent,93.994,4.857,no,2022
1,1,57,services,married,high.school,unknown,no,May,19,149,1,0,nonexistent,93.994,4.857,no,2022
2,2,37,services,married,high.school,no,yes,May,23,226,1,0,nonexistent,93.994,4.857,no,2022
3,3,40,admin.,married,basic.6y,no,no,May,27,151,1,0,nonexistent,93.994,4.857,no,2022
4,4,56,services,married,high.school,no,no,May,3,307,1,0,nonexistent,93.994,4.857,no,2022


In [362]:
df['date'] = df["year"] + "-" +df["month"] + "-"+df["day"] 
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year,date
0,0,56,housemaid,married,basic.4y,no,no,May,13,261,1,0,nonexistent,93.994,4.857,no,2022,2022-May-13
1,1,57,services,married,high.school,unknown,no,May,19,149,1,0,nonexistent,93.994,4.857,no,2022,2022-May-19
2,2,37,services,married,high.school,no,yes,May,23,226,1,0,nonexistent,93.994,4.857,no,2022,2022-May-23
3,3,40,admin.,married,basic.6y,no,no,May,27,151,1,0,nonexistent,93.994,4.857,no,2022,2022-May-27
4,4,56,services,married,high.school,no,no,May,3,307,1,0,nonexistent,93.994,4.857,no,2022,2022-May-3


In [363]:
campaign["last_contact_date"] = pd.to_datetime(df['date'],format="%Y-%b-%d")
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [364]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [365]:
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [366]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB


In [367]:
economics.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [368]:
economics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB


In [369]:
client.to_csv("client.csv",index=False)
campaign.to_csv("campaign.csv",index=False)
economics.to_csv("economics.csv",index=False)